<a href="https://colab.research.google.com/github/trehansalil/llm_series/blob/main/Advanced%20RAG%20Pipeline%20using%20OpenAI%20Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cassio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 67.2 MB/s eta 0:00:00


In [3]:
import os
import cassio
from getpass import getpass
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ASTRA_DB_ENDPOINT'] = userdata.get('ASTRA_DB_ENDPOINT')
os.environ['ASTRA_DB_APPLICATION_TOKEN'] =  userdata.get('ASTRA_DB_APPLICATION_TOKEN')

## Setup for RAG Pipeline

In [4]:
!pip install -q astrapy langchain_astradb langchain openai ragstack-ai datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [5]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra

embedding = OpenAIEmbeddings()

vstore = AstraDBVectorStore(
    collection_name = "test",
    embedding = embedding,
    token=os.getenv('ASTRA_DB_APPLICATION_TOKEN'),
    api_endpoint=os.getenv('ASTRA_DB_ENDPOINT')
)

print("Astra vector store is configured")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Astra vector store is configured


In [6]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient(os.environ['ASTRA_DB_APPLICATION_TOKEN'])
db = client.get_database_by_api_endpoint(
  "https://c1ba1638-a6e8-412a-8ed8-fe286299f806-us-east-2.apps.astra.datastax.com"
)

print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: ['test']


In [7]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")['train']
print("an Example entry:")
print(philo_dataset[16])

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

an Example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [8]:
from langchain.schema import Document
docs = []

for entry in philo_dataset:
    metadata = {"author": entry['author']}
    if entry['tags']:
        # Add metadata tags to the metadata dictionary
        for tag in entry['tags'].split(";"):
            metadata[tag] = 'y'
    # Create a Laangchain Document with the quote and metadata tags
    doc = Document(page_content=entry['quote'], metadata=metadata)
    docs.append(doc)

In [9]:
# Create Embeddings by inserting your documents into the vector store

inserted_ids = vstore.add_documents(docs)

In [10]:
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents


## Basic Retrieval

In [12]:
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '9d36d2352d264c79a6c48c8fd9d9c238', 'content': 'Big fish eat small fish with as much right as they have power.', '$vector': [0.015496551895401528, -0.03369275662319513, 0.02071059234438798, 0.009468644437025006, -0.0224441935312277, 0.007417435692409494, -0.021676645852585583, -0.019466632164928895, -0.0011794450165123864, -0.05018183114711572, 0.02742003052377354, 0.003592923710286671, 0.02386018948859819, -0.023251443493368062, -0.009428944016172008, 0.016118531053808446, 0.031866522362795247, -0.021676645852585583, 0.03199885585720841, -0.011043443009130112, 0.019215193682666245, 0.02292060416939942, 0.0060775329844427696, -0.00878049666876626, 0.007000577082501209, 0.017124284982859048, 0.005204115343773201, -0.008998851777425619, 0.013432109521947913, -0.03022555424951569, 0.04470311719004458, 0.0005553978562169428, -0.0012605009349381051, 0.006901325099046091, -0.00978625059781686, -0.002410170286564759, 0.0014904347121311734, -0.00715276358130874,

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you dont know the answer, say you don't know the answer.
Context: {context}
Question: {question}
You answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subjects are philosophers most concerned with?")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'Philosophers are most concerned with truth, unchanging reality, the possibility, principles, and extent of human knowledge, and wonder.'

In [15]:
chain.invoke("In the given context, what are the highest, most varied & lasting pleasures?")

'The highest, most varied and lasting pleasures are those of the mind.'

In [16]:
chain.invoke("In the given context, what difficulty are philosophers most concerned with?")

'The difficulty philosophers are most concerned with in the given context is determining the possibility, principles, and extent of human knowledge a priori.'

In [18]:
chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

"The most important thing to allow the brain is the full measure of sleep required to restore it. The tags for the documents in the context are 'author': 'schopenhauer', 'ethics': 'y', 'knowledge': 'y' and 'author': 'sartre'."